In [5]:
## Standard libraries
import os
import math
import numpy as np 
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
# Set Seaborn style
sns.set(style='darkgrid', font_scale=1.2)
from sklearn.datasets import make_moons

## Progress bar
from tqdm.notebook import tqdm

import torch
print("Using torch", torch.__version__)
#torch.manual_seed(42) # Setting the seed
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.distributions.multivariate_normal import MultivariateNormal

from torch.utils.tensorboard import SummaryWriter
import torch.profiler

from GPUtil import showUtilization as gpu_usage

Using torch 2.1.0


In [6]:
class langevin_sampler():
    def __init__(self, sampling_steps, sampling_noise, device):
        self.device = device
        self.K = torch.tensor(sampling_steps, device=self.device)
        self.s = torch.tensor(sampling_noise, device=self.device)
    
    def get_sample(self, initial_sample, model, x):
        x_k = initial_sample
        z_k = x
        step=0
        
        while step < self.K:
            # Compute gradient
            grad = model.grad_log_fn(x_k, z_k)
            
            # Update sample
            x_k = x_k + self.s * grad + (torch.sqrt(2*self.s)*torch.randn_like(x_k, device=self.device))  
            
            step += 1             
        
        return x_k

class GaussianMixtureModel(nn.Module):
    def __init__(self, num_components, dim, device):
        super(GaussianMixtureModel, self).__init__()
        self.num_components = num_components
        self.dim = dim
        self.device = device

        self.weights = nn.Parameter(torch.ones(num_components, device=self.device) / num_components)
        self.means = nn.Parameter(torch.randn(num_components, dim, device=self.device))
        self.variances = nn.Parameter(torch.ones(num_components, dim, device=self.device))

    def log_likelihood(self, x):
        log_probs = []
        for i in range(self.num_components):
            log_prob = -0.5 * torch.sum(torch.log(2 * torch.tensor(3.1416, device=self.device) * self.variances[i]) +
                                       (x - self.means[i]) ** 2 / self.variances[i], dim=1)
            log_probs.append(torch.log(self.weights[i]) + log_prob)
        log_probs = torch.stack(log_probs, dim=1)
        return torch.logsumexp(log_probs, dim=1)

    def grad_log_fn(self, x, z):
        log_probs = []
        for i in range(self.num_components):
            log_prob = -0.5 * torch.sum(torch.log(2 * torch.tensor(3.1416, device=self.device) * self.variances[i]) +
                                       (x - self.means[i]) ** 2 / self.variances[i], dim=1)
            log_probs.append(torch.log(self.weights[i]) + log_prob)
        log_probs = torch.stack(log_probs, dim=1)

        probs = torch.exp(log_probs - torch.logsumexp(log_probs, dim=1, keepdim=True))

        grad_weights = probs
        grad_means = probs.unsqueeze(-1) * (x.unsqueeze(1) - self.means)
        grad_variances = probs.unsqueeze(-1) * (((x.unsqueeze(1) - self.means) ** 2 / self.variances) - 1)

        return torch.cat((torch.sum(grad_weights, dim=0), torch.sum(grad_means, dim=0), torch.sum(grad_variances, dim=0)), dim=0)

In [7]:
# Parameters
NUM_EPOCHS = 500
BATCH_SIZE = 128
LR = 1e-4
SAMPLE_STEPS = 20
SAMPLES = 500
SAMPLE_NOISE = 0.5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate 'make_moons' data
X, _ = make_moons(n_samples=SAMPLES, noise=0.05, random_state=42)
X = torch.tensor(X).float().to(device)

# Create the Gaussian Mixture Model
gmm = GaussianMixtureModel(num_components=2, dim=2, device=device)
sampler = langevin_sampler(SAMPLE_STEPS, SAMPLE_NOISE, device)
loss_fn = nn.MSELoss()  # Define your loss function
optimizer = torch.optim.Adam(gmm.parameters(), lr=LR)  # Define your optimizer

# Training loop
for epoch in range(NUM_EPOCHS):
    # Sample from the GMM using the Langevin sampler
    sample = sampler.get_sample(torch.randn(1, 2, device=device), gmm, X)

    # Compute the loss
    loss = loss_fn(sample, X)  # Define your loss function

    # Perform backpropagation
    loss.backward()

    # Update the parameters
    optimizer.step()
    optimizer.zero_grad()
    # Printing progress
    if epoch % 50 == 0:
        print(f"Epoch [{epoch}/{NUM_EPOCHS}], Loss: ...")  # Update loss calculation here

# Plot the generated samples
plt.scatter(X[:, 0].cpu(), X[:, 1].cpu(), color='blue', label='Original Data')
plt.scatter(sample[:, 0].cpu(), sample[:, 1].cpu(), color='red', label='Generated Samples')
plt.legend()
plt.show()

RuntimeError: Tensors must have same number of dimensions: got 1 and 2